# Friday 3.2
let's try this again but rip off ChatGPT

## Imports

In [10]:
import re
from transformers import AutoTokenizer, AutoModelForCausalLM,  AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import torch

## Load base model

In [17]:
# Set the model and tokenizer names
model_name = "bigscience/T0_3B"
tokenizer_name = "bigscience/T0_3B"

# Load the model and tokenizer
model =  AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
#model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")

loading configuration file https://huggingface.co/bigscience/T0_3B/resolve/main/config.json from cache at /home/vscode/.cache/huggingface/transformers/7b128e6b48089ae556964fea17b39635abd0124e77f8fa30267896af500a4d6d.a54ecffc6881ea8ae0af8a0dca40a7bcd51ccf51d434d2f7d0569844f6fb1c60
Model config T5Config {
  "_name_or_path": "bigscience/T0_3B",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 5120,
  "d_kv": 64,
  "d_model": 2048,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 24,
  "num_heads": 32,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "transfo

ValueError: Unrecognized configuration class <class 'transformers.models.t5.configuration_t5.T5Config'> for this kind of AutoModel: AutoModelForCausalLM.
Model type should be one of BartConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BlenderbotConfig, BlenderbotSmallConfig, BloomConfig, CamembertConfig, CTRLConfig, Data2VecTextConfig, ElectraConfig, GPT2Config, GPTNeoConfig, GPTNeoXConfig, GPTJConfig, MarianConfig, MBartConfig, MegatronBertConfig, OpenAIGPTConfig, OPTConfig, PegasusConfig, PLBartConfig, ProphetNetConfig, QDQBertConfig, ReformerConfig, RemBertConfig, RobertaConfig, RoFormerConfig, Speech2Text2Config, TransfoXLConfig, TrOCRConfig, XGLMConfig, XLMConfig, XLMProphetNetConfig, XLMRobertaConfig, XLMRobertaXLConfig, XLNetConfig.

## Level fine tunning
Do a statndard fine tune on the model

### The finetune function

In [4]:
def fine_tune(model, train_dataloader, val_dataloader, epochs=2):
  # Set up the trainer
  training_args = TrainingArguments(
      output_dir="./results",          # output directory
      num_train_epochs=epochs,         # number of epochs
      per_device_train_batch_size=16,  # batch size
      per_device_eval_batch_size=64,   # evaluation batch size
      warmup_steps=500,                # number of warmup steps
      weight_decay=0.01,               # L2 weight decay
      logging_dir="./logs"             # logging directory
  )

  trainer = Trainer(
      model=model,                     # the model to train
      args=training_args,              # training arguments
      train_dataset=train_dataloader,  # training dataset
      eval_dataset=val_dataloader      # validation dataset
  )

  # Fine-tune the model
  trainer.train()

### Data loader

In [6]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
dataset = load_dataset("Anthropic/hh-rlhf")
def tokenize_function(examples):
    return tokenizer(examples["chosen"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Using pad_token, but it is not set yet.
Using custom data configuration Anthropic--hh-rlhf-03696fd7b9f56a9e
Reusing dataset json (/home/vscode/.cache/huggingface/datasets/Anthropic___json/Anthropic--hh-rlhf-03696fd7b9f56a9e/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/161 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

In [7]:
train_dataloader = tokenized_datasets["train"].shuffle(seed=42)
val_dataloader = tokenized_datasets["test"].shuffle(seed=42)

### Tune it!

In [8]:
fine_tune(model, train_dataloader, val_dataloader)

The following columns in the training set don't have a corresponding argument in `GPTNeoForCausalLM.forward` and have been ignored: rejected, chosen. If rejected, chosen are not expected by `GPTNeoForCausalLM.forward`,  you can safely ignore this message.
/home/vscode/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 160800
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 20100


IndexError: index out of range in self

In [16]:
def generate_response_single(prompt):
  # Encode the prompt and generate a response
  input_ids = tokenizer.encode(prompt, return_tensors='pt')
  response = model.generate(input_ids, max_length=1024, top_p=0.9, top_k=20)
  print(tokenizer.decode(response[0]))
generate_response_single("Human: Hello there Assistant: ")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Human: Hello there Assistant: 

I am a newbie in the field of programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the basics of programming. I am new to programming and I am trying to understand the b

## Chatbot interface

In [12]:
def chatbot():
  step = 0
  chat_history = []
  while(True):
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("Friday: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
    step = step + 1

# Run the chatbot
chatbot()

Friday: Q:

How to get the value of a variable in a function?

I have a function that takes a variable and returns a value.
function get_value(var) {
    var value = 0;
    return value;
}

I want to get the value of the variable in the function.
function get_value(var) {
    var value = 0;
    return value;
}

I have tried to use the function like this:
function get_value(var) {
    var value = 0;
    return value;
}

But it doesn't work.

A:

You can use the function like this:
function get_value(var) {
    var value = 0;
    return value;
}

function get_value(var) {
    var value = 0;
    return value;
}

function get_value(var) {
    var value = 0;
    return value;
}

function get_value(var) {
    var value = 0;
    return value;
}

function get_value(var) {
    var value = 0;
    return value;
}

function get_value(var) {
    var value = 0;
    return value;
}

function get_value(var) {
    var value = 0;
    return value;
}

function get_value(var) {
    var value = 0;
    retu

Input length of input_ids is 1001, but ``max_length`` is set to 1000. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


KeyboardInterrupt: 